# 1. MLP 설계할 떄 dropout 적용하기

In [1]:
''' 1. Module Import '''
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
''' 2. 딥러닝 모델을 설계할 때 활용하는 장비 확인 '''
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

Using PyTorch version: 1.9.0+cpu  Device: cpu


In [3]:
BATCH_SIZE = 32 # 배치 사이즈 32
EPOCHS = 10 # 10번의 에폭 반복

In [4]:
''' 3. MNIST 데이터 다운로드 (Train set, Test set 분리하기) '''
train_dataset = datasets.MNIST(root = "../data/MNIST",
                               train = True,
                               download = True,
                               transform = transforms.ToTensor())

test_dataset = datasets.MNIST(root = "../data/MNIST",
                              train = False,
                              transform = transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = BATCH_SIZE,
                                          shuffle = False)

C:\Users\tjdqj\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
''' 4. 데이터 확인하기 (1) '''
for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

X_train: torch.Size([32, 1, 28, 28]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor


In [6]:
''' 6. Multi Layer Perceptron (MLP) 모델 설계하기 '''
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.dropout_prob = 0.5 # Dropout 생성

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.fc1(x)
        x = F.sigmoid(x) # 시그모이드 함수
        x = F.dropout(x, training = self.training, p = self.dropout_prob) # Dropout 생성
        x = self.fc2(x)
        x = F.sigmoid(x)
        x = F.dropout(x, training = self.training, p = self.dropout_prob) # Dropout 생성
        x = self.fc3(x)
        x = F.log_softmax(x, dim = 1) 
        return x

In [7]:
''' 7. Optimizer, Objective Function 설정하기 '''
model = Net().to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5) # 학습 SGD(momentum), lr = 0.01 지정
criterion = nn.CrossEntropyLoss() # binary 이니까 crossEntropyLoss

print(model)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)


In [8]:
''' 8. MLP 모델 학습을 진행하며 학습 데이터에 대한 모델 성능을 확인하는 함수 정의 '''
def train(model, train_loader, optimizer, log_interval):
    model.train()
    
    # 확인하는 구간 정의
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item()))

In [9]:
''' 9. 학습되는 과정 속에서 검증 데이터에 대한 모델 성능을 확인하는 함수 정의 '''
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    # no_grad로 초기화, 검증데이터 확인
    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
    
    test_loss /= (len(test_loader.dataset) / BATCH_SIZE)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [10]:
''' 10. MLP 학습 실행하며 Train, Test set의 Loss 및 Test set Accuracy 확인하기 '''
for epoch in range(1, EPOCHS + 1):
    # 모델 학습
    train(model, train_loader, optimizer, log_interval = 200)
    # 테스트 검증
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(epoch, test_loss, test_accuracy))

C:\Users\tjdqj\anaconda3\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [0/60000 (0%)]	Train Loss: 2.372763
Train Epoch: 1 [6400/60000 (11%)]	Train Loss: 2.270722
Train Epoch: 1 [12800/60000 (21%)]	Train Loss: 2.347056
Train Epoch: 1 [19200/60000 (32%)]	Train Loss: 2.392092
Train Epoch: 1 [25600/60000 (43%)]	Train Loss: 2.347405
Train Epoch: 1 [32000/60000 (53%)]	Train Loss: 2.285298
Train Epoch: 1 [38400/60000 (64%)]	Train Loss: 2.355855
Train Epoch: 1 [44800/60000 (75%)]	Train Loss: 2.281067
Train Epoch: 1 [51200/60000 (85%)]	Train Loss: 2.327244
Train Epoch: 1 [57600/60000 (96%)]	Train Loss: 2.275053

[EPOCH: 1], 	Test Loss: 2.2809, 	Test Accuracy: 11.04 % 

Train Epoch: 2 [0/60000 (0%)]	Train Loss: 2.302710
Train Epoch: 2 [6400/60000 (11%)]	Train Loss: 2.303772
Train Epoch: 2 [12800/60000 (21%)]	Train Loss: 2.282429
Train Epoch: 2 [19200/60000 (32%)]	Train Loss: 2.254463
Train Epoch: 2 [25600/60000 (43%)]	Train Loss: 2.214751
Train Epoch: 2 [32000/60000 (53%)]	Train Loss: 2.282789
Train Epoch: 2 [38400/60000 (64%)]	Train Loss: 2.282966
T

# 2. ReLU 함수 적용

In [10]:
''' 6. Multi Layer Perceptron (MLP) 모델 설계하기 '''
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.dropout_prob = 0.5

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.fc1(x)
        x = F.relu(x)       # sigmoid 대신 ReLU가 들어갔다!!
        x = F.dropout(x, training = self.training, p = self.dropout_prob)
        x = self.fc2(x)
        x = F.relu(x)       # sigmoid 대신 ReLU가 들어갔다!!
        x = F.dropout(x, training = self.training, p = self.dropout_prob)
        x = self.fc3(x)
        x = F.log_softmax(x, dim = 1)
        return x

In [11]:
''' 10. MLP 학습 실행하며 Train, Test set의 Loss 및 Test set Accuracy 확인하기 '''
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))

C:\Users\tjdqj\anaconda3\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [0/60000 (0%)]	Train Loss: 2.413355
Train Epoch: 1 [6400/60000 (11%)]	Train Loss: 2.258033
Train Epoch: 1 [12800/60000 (21%)]	Train Loss: 2.402081
Train Epoch: 1 [19200/60000 (32%)]	Train Loss: 2.284401
Train Epoch: 1 [25600/60000 (43%)]	Train Loss: 2.244243
Train Epoch: 1 [32000/60000 (53%)]	Train Loss: 2.324172
Train Epoch: 1 [38400/60000 (64%)]	Train Loss: 2.236053
Train Epoch: 1 [44800/60000 (75%)]	Train Loss: 2.273867
Train Epoch: 1 [51200/60000 (85%)]	Train Loss: 2.237538
Train Epoch: 1 [57600/60000 (96%)]	Train Loss: 2.265991

[EPOCH: 1], 	Test Loss: 2.2818, 	Test Accuracy: 18.07 % 

Train Epoch: 2 [0/60000 (0%)]	Train Loss: 2.318339
Train Epoch: 2 [6400/60000 (11%)]	Train Loss: 2.320609
Train Epoch: 2 [12800/60000 (21%)]	Train Loss: 2.337325
Train Epoch: 2 [19200/60000 (32%)]	Train Loss: 2.240350
Train Epoch: 2 [25600/60000 (43%)]	Train Loss: 2.292237
Train Epoch: 2 [32000/60000 (53%)]	Train Loss: 2.208195
Train Epoch: 2 [38400/60000 (64%)]	Train Loss: 2.200290
T

# 3. Batch Normalization 적용

In [12]:
''' 6. Multi Layer Perceptron (MLP) 모델 설계하기 '''
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.dropout_prob = 0.5
        self.batch_norm1 = nn.BatchNorm1d(512)
        self.batch_norm2 = nn.BatchNorm1d(256)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.fc1(x)
        x = self.batch_norm1(x)
        x = F.relu(x)
        x = F.dropout(x, training = self.training, p = self.dropout_prob)
        x = self.fc2(x)
        x = self.batch_norm2(x)
        x = F.relu(x)
        x = F.dropout(x, training = self.training, p = self.dropout_prob)
        x = self.fc3(x)
        x = F.log_softmax(x, dim = 1)
        return x

In [13]:
''' 10. MLP 학습 실행하며 Train, Test set의 Loss 및 Test set Accuracy 확인하기 '''
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Train Loss: 0.739340
Train Epoch: 1 [6400/60000 (11%)]	Train Loss: 0.307793
Train Epoch: 1 [12800/60000 (21%)]	Train Loss: 0.473834
Train Epoch: 1 [19200/60000 (32%)]	Train Loss: 0.300209
Train Epoch: 1 [25600/60000 (43%)]	Train Loss: 0.774276
Train Epoch: 1 [32000/60000 (53%)]	Train Loss: 0.530995
Train Epoch: 1 [38400/60000 (64%)]	Train Loss: 0.779925
Train Epoch: 1 [44800/60000 (75%)]	Train Loss: 0.464488
Train Epoch: 1 [51200/60000 (85%)]	Train Loss: 0.547864
Train Epoch: 1 [57600/60000 (96%)]	Train Loss: 0.499111

[EPOCH: 1], 	Test Loss: 0.4206, 	Test Accuracy: 87.58 % 

Train Epoch: 2 [0/60000 (0%)]	Train Loss: 0.659648
Train Epoch: 2 [6400/60000 (11%)]	Train Loss: 0.536110
Train Epoch: 2 [12800/60000 (21%)]	Train Loss: 1.092418
Train Epoch: 2 [19200/60000 (32%)]	Train Loss: 0.374606
Train Epoch: 2 [25600/60000 (43%)]	Train Loss: 0.386496
Train Epoch: 2 [32000/60000 (53%)]	Train Loss: 0.654428
Train Epoch: 2 [38400/60000 (64%)]	Train Loss: 0.374927
T

# 4. He Initialization 초기화 적용

In [14]:
''' 7. Optimizer, Objective Function 설정하기 '''
import torch.nn.init as init
def weight_init(m):
    if isinstance(m, nn.Linear):
        init.kaiming_uniform_(m.weight.data) # kaiming_uniform_는 He 초깃값이다.

model = Net().to(DEVICE)
model.apply(weight_init)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)
criterion = nn.CrossEntropyLoss()

print(model)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
  (batch_norm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [15]:
''' 10. MLP 학습 실행하며 Train, Test set의 Loss 및 Test set Accuracy 확인하기 '''
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Train Loss: 3.539637
Train Epoch: 1 [6400/60000 (11%)]	Train Loss: 0.650326
Train Epoch: 1 [12800/60000 (21%)]	Train Loss: 0.528997
Train Epoch: 1 [19200/60000 (32%)]	Train Loss: 0.491456
Train Epoch: 1 [25600/60000 (43%)]	Train Loss: 0.382745
Train Epoch: 1 [32000/60000 (53%)]	Train Loss: 1.023613
Train Epoch: 1 [38400/60000 (64%)]	Train Loss: 0.575118
Train Epoch: 1 [44800/60000 (75%)]	Train Loss: 0.476374
Train Epoch: 1 [51200/60000 (85%)]	Train Loss: 0.642743
Train Epoch: 1 [57600/60000 (96%)]	Train Loss: 0.254190

[EPOCH: 1], 	Test Loss: 0.2224, 	Test Accuracy: 93.29 % 

Train Epoch: 2 [0/60000 (0%)]	Train Loss: 0.396919
Train Epoch: 2 [6400/60000 (11%)]	Train Loss: 0.254240
Train Epoch: 2 [12800/60000 (21%)]	Train Loss: 0.412769
Train Epoch: 2 [19200/60000 (32%)]	Train Loss: 0.370537
Train Epoch: 2 [25600/60000 (43%)]	Train Loss: 0.179562
Train Epoch: 2 [32000/60000 (53%)]	Train Loss: 0.428764
Train Epoch: 2 [38400/60000 (64%)]	Train Loss: 0.437366
T

# 5. Optimizer 적용

In [21]:
''' 7. Optimizer, Objective Function 설정하기 '''
import torch.nn.init as init
def weight_init(m):
    if isinstance(m, nn.Linear):
        init.kaiming_uniform_(m.weight.data) # kaiming_uniform_는 He 초깃값이다.

model = Net().to(DEVICE)
model.apply(weight_init)
# optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
criterion = nn.CrossEntropyLoss()

print(model)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
  (batch_norm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [22]:
''' 10. MLP 학습 실행하며 Train, Test set의 Loss 및 Test set Accuracy 확인하기 '''
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Train Loss: 2.665775
Train Epoch: 1 [6400/60000 (11%)]	Train Loss: 0.681709
Train Epoch: 1 [12800/60000 (21%)]	Train Loss: 0.209609
Train Epoch: 1 [19200/60000 (32%)]	Train Loss: 0.134814
Train Epoch: 1 [25600/60000 (43%)]	Train Loss: 0.511366
Train Epoch: 1 [32000/60000 (53%)]	Train Loss: 0.533447
Train Epoch: 1 [38400/60000 (64%)]	Train Loss: 0.386680
Train Epoch: 1 [44800/60000 (75%)]	Train Loss: 0.246109
Train Epoch: 1 [51200/60000 (85%)]	Train Loss: 0.219392
Train Epoch: 1 [57600/60000 (96%)]	Train Loss: 0.434873

[EPOCH: 1], 	Test Loss: 0.1246, 	Test Accuracy: 96.01 % 

Train Epoch: 2 [0/60000 (0%)]	Train Loss: 0.482386
Train Epoch: 2 [6400/60000 (11%)]	Train Loss: 0.257432
Train Epoch: 2 [12800/60000 (21%)]	Train Loss: 0.324007
Train Epoch: 2 [19200/60000 (32%)]	Train Loss: 0.118995
Train Epoch: 2 [25600/60000 (43%)]	Train Loss: 0.212950
Train Epoch: 2 [32000/60000 (53%)]	Train Loss: 0.143802
Train Epoch: 2 [38400/60000 (64%)]	Train Loss: 0.358522
T